In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Importing the txt file for the project
text = []
with open(r'C:\Users\Zamani\Downloads\ATT00001.txt') as f:
    text = f.read()
count = 0
for i in text:
    count += 1
#print (text)    
print('Total letter count is ', count)

Total letter count is  19962


In [3]:
punctuations = ["1","2","3","4","5","6","7","8","9","0","!",",","`", "~","!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "_", "-", "=", "+", "'", ":", ";", ".", "/", "<", ">", "?", "[", "]", "{", "}", "é"]

In [4]:
punct=set(punctuations)
stop_words = ["efwjjrysbjf"]

In [5]:
def RemovePunctuation(token):
  for p in punctuations:
    token = token.replace(p, '')
    return token

def letters_only(astr):
  return astr.isalpha()

In [6]:
#new_document = []
document = open("ATT00001.txt", "r", errors='ignore')
raw_document = document.read(-1)

In [7]:
#raw_document

In [8]:
#Text editing to remove all unwanted characters
txtf = ' '.join([word for word in raw_document.split() if word not in stop_words])

txt = txtf.lower()

# raw_document.split()

In [9]:
punct = set(punctuations) 
textp = ''.join([ch for ch in txt if ch not in punct])
#print(textp)

In [10]:
len(textp)

17340

In [11]:
N = 3
M = 27
matrix = np.random.rand(N,N)
transition = matrix/matrix.sum(axis=1)[:,None]  
#transition
init_probs = [0, 0, 1]
print(init_probs)

[0, 0, 1]


In [12]:
matrixE = np.random.rand(N,M)
emission = matrixE/matrixE.sum(axis=1)[:,None]
#emission

In [13]:
#defining states and sequence symbols
states = ['H', 'C', 'T']
states_dic = {'H':0, 'C':1}
sequence = ['a','b','c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']

# sequence = (list(set(textp))).sort() #[' ','a','b','c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
#sequence = sequencee.sort()
sequence_syms = {'a':0,'b':1,'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7, 'i':8, 'j':9, 'k':10, 'l':11, 'm':12, 'n':13, 
                 'o':14, 'p':15, 'q':16, 'r':17, 's':18, 't':19, 'u':20, 'v':21, 'w':22, 'x':23, 'y':24, 'z':25, ' ':26}
# len(sequence_syms)

In [14]:
#test sequence
test_sequence = textp
test_sequence = [x for x in test_sequence]
# test_sequence

In [15]:
#probabilities of going to end state
end_probs = transition[-1] #[0.6, 0.3, 0.1]
#probabilities of going from start state
start_probs = transition[0] # [0.4, 0.3, 0.3]

In [16]:
#function to find forward probabilities
def forward_probs():
    # node values stored during forward algorithm
    rndmatF = np.random.rand(N, len(test_sequence))
    node_values_fwd = rndmatF/rndmatF.sum(axis=1)[:, None]  
    #node_values_fwd np.zeros((len(states), len(test_sequence)))
    
    for i, sequence_val in enumerate(test_sequence):
        for j in range(len(states)):
            # if first sequence value then do this
            if (i == 0):
                node_values_fwd[j, i] = init_probs[j] * emission[j, sequence_syms[sequence_val]]
            # else perform this
            else:
                values = [node_values_fwd[k, i - 1] * emission[j, sequence_syms[sequence_val]] * transition[k, j] for k in
                          range(len(states))]
                node_values_fwd[j, i] = sum(values)
                #print( values)
    #end state value
    end_state = np.multiply(node_values_fwd[:, 0], end_probs)
    # end_state = np.multiply(node_values_fwd[:, -1], end_probs)
    end_state_val = sum(end_state)
    return node_values_fwd, end_state_val
    
   # print(node_values_fwd)
    #alpha = forward_probs()
    #print(alpha )

In [17]:
#function to find backward probabilities
def backward_probs():
    # node values stored during forward algorithm
    rndmatB = np.random.rand(N, len(test_sequence))
    node_values_bwd = rndmatB/rndmatB.sum(axis=1)[:, None]
    #node_values_bwd = np.zeros((len(states), len(test_sequence)))

    #for i, sequence_val in enumerate(test_sequence):
    for i in range(1,len(test_sequence)+1):
        for j in range(len(states)):
            # if first sequence value then do this
            if (-i == -1):
                node_values_bwd[j, -i] = end_probs[j]
            # else perform this
            else:
                values = [node_values_bwd[k, -i+1] * emission[k, sequence_syms[test_sequence[-i+1]]] * transition[j, k] for k in range(len(states))]
                node_values_bwd[j, -i] = sum(values)

    #start state value
    start_state = [node_values_bwd[m, -1] * emission[m, sequence_syms[test_sequence[0]]] for m in range(len(states))]
    # start_state = [node_values_bwd[m, 0] * emission[m, sequence_syms[test_sequence[0]]] for m in range(len(states))]
    start_state = np.multiply(start_state, start_probs)
    start_state_val = sum(start_state)
    return node_values_bwd, start_state_val

    #beta = backward_probs()



In [18]:
#function to find si probabilities
def si_probs(forward, backward, forward_val):
    rndmatS = np.random.rand(N, len(test_sequence) - 1, N)
    si_probabilities = rndmatS/rndmatS.sum(axis=1)[:, None]
    # si_probabilities = np.zeros((len(states), len(test_sequence)-1, len(states)))

    for i in range(len(test_sequence)-1):
        for j in range(len(states)):
            for k in range(len(states)):
                si_probabilities[j,i,k] = si_probabilities[j, i, k] + ((forward[j,i] * backward[k,i+1] * transition[j,k] * emission[k,sequence_syms[test_sequence[i+1]]] ) / forward_val)
                # si_probabilities[j,i,k] = ( forward[j,i] * backward[k,i+1] * transition[j,k] * emission[k,sequence_syms[test_sequence[i+1]]] ) \
                #                                     / forward_val                                    
    return si_probabilities

In [19]:
#function to find gamma probabilities
def gamma_probs(forward, backward, forward_val):

    rndmatG = np.random.rand(N, len(test_sequence))
    gamma_probabilities = rndmatG/rndmatG.sum(axis=1)[:, None]
    # gamma_probabilities = np.zeros((len(states), len(test_sequence)))

    for i in range(len(test_sequence)):
        for j in range(len(states)):
            #gamma_probabilities[j,i] = ( forward[j,i] * backward[j,i] * emission[j,sequence_syms[test_sequence[i]]] ) / forward_val
            gamma_probabilities[j, i] = gamma_probabilities[j, i] + ((forward[j, i] * backward[j, i]) / forward_val)

    return gamma_probabilities


In [20]:
#performing iterations until convergence

for iteration in range(10):

  # print('\nIteration No: ', iteration + 1)
  # print('\nTransition:\n ', transition)
  # print('\nEmission: \n', emission)

  #Calling probability functions to calculate all probabilities
  fwd_probs, fwd_val = forward_probs()
  bwd_probs, bwd_val = backward_probs()
  si_probabilities = si_probs(fwd_probs, bwd_probs, fwd_val)
  gamma_probabilities = gamma_probs(fwd_probs, bwd_probs, fwd_val)

  
  # caclculating 'a' and 'b' matrices
  a = np.zeros((len(states), len(states)))
  # a = np.zeros((ne, ne))
  b = np.zeros((len(states), len(sequence_syms)))
  # b = np.zeros((ne, me))

  #'a' matrix
  for j in range(len(states)):
    for i in range(len(states)):
      for t in range(len(test_sequence)-1):
        a[j,i] = a[j, i] + si_probabilities[j, t, i]

      denomenator_a = [si_probabilities[j, t_x, i_x] for t_x in range(len(test_sequence) - 1) for i_x in range(len(states))]
      denomenator_a = sum(denomenator_a)

      if (denomenator_a == 0):
        a[j, i] = 0
      else:
        a[j, i] = a[j, i]/denomenator_a

  #'b' matrix
  for j in range(len(states)): #states
    for i in range(len(sequence)): #seq
      indices = [idx for idx, val in enumerate(test_sequence) if val == sequence[i]]
      numerator_b = sum( gamma_probabilities[j, indices] )
      denomenator_b = sum( gamma_probabilities[j,:] )

      if (denomenator_b == 0):
        b[j, i] = 0
      else:
        b[j, i] = numerator_b / denomenator_b


print('\nMatrix bjk:\n')
print(np.matrix(b.round(decimals=4)))
  # maximums = [[0 for x in range(len(b[0]))] for x in range(len(b))]
  
  #print(np.matrix(b))
transition = a
emission = b

print('\nMatrix aij:\n')
print(np.matrix(a.round(decimals=4)))
# print(np.matrix(a))

new_fwd_temp, new_fwd_temp_val = forward_probs()
print('\nNew forward probability: ', new_fwd_temp_val)
diff =  np.abs(fwd_val - new_fwd_temp_val)
print('Difference in forward probability: ', diff)
print('\nIteration No: ', iteration + 1)
result = (b.max(axis=0))
print(result)

print("\nSorted Array in ascending order is:\n")
print(np.sort(result))
  #if (diff < 0.0000001):
      #break

c = 1




Matrix bjk:

[[0.0657 0.0131 0.0304 0.0311 0.1081 0.0154 0.014  0.0442 0.0556 0.0018
  0.0051 0.036  0.024  0.0508 0.0615 0.0177 0.0006 0.0555 0.0585 0.0727
  0.0236 0.0081 0.0148 0.0025 0.0173 0.0006 0.1712]
 [0.0652 0.0145 0.0317 0.0309 0.1107 0.0144 0.014  0.0449 0.0556 0.0018
  0.0046 0.036  0.0243 0.0517 0.0614 0.0184 0.0007 0.053  0.0577 0.0721
  0.0246 0.0085 0.0151 0.002  0.0176 0.0006 0.1682]
 [0.065  0.0144 0.0303 0.0314 0.1106 0.0151 0.0146 0.0449 0.0537 0.0017
  0.0046 0.0349 0.0229 0.0498 0.0606 0.0183 0.0005 0.0553 0.057  0.0753
  0.0251 0.0088 0.0149 0.0024 0.018  0.0006 0.1692]]

Matrix aij:

[[0.3333 0.3333 0.3333]
 [0.3333 0.3333 0.3333]
 [0.3333 0.3333 0.3333]]

New forward probability:  0.01667255936978779
Difference in forward probability:  0.005375067341187444

Iteration No:  10
[0.0656533  0.01445203 0.03168166 0.03140707 0.11065753 0.01544806
 0.01458822 0.04488236 0.05562349 0.00178058 0.00506633 0.03603454
 0.02428943 0.05166739 0.0615311  0.01836035 0.000722

In [21]:
print(np.sort(result))

#wt = 1/new_fwd_temp_val
#print('\nScaling value: ', wt)
#plt(result)

[0.00064785 0.00072216 0.00178058 0.00253836 0.00506633 0.00878263
 0.01445203 0.01458822 0.01510837 0.01544806 0.01804987 0.01836035
 0.02428943 0.02514157 0.03140707 0.03168166 0.03603454 0.04488236
 0.05166739 0.0555435  0.05562349 0.05849295 0.0615311  0.0656533
 0.07528267 0.11065753 0.17124753]


In [260]:
wt = 1/fwd_val
print('\nScaling value: ', wt)

alpha = wt * aij[0]
alp_val = sum(alpha)
print('\nalpha_value ', alp_val)

beta = wt * bjk[1]
bet_val = sum(beta)
bet_val


Scaling value:  1196.431480316341

alpha_value  1196.4314803163534


1196.4314803163413